In [ ]:
# Prepare inputs for hyperbolics.
import pickle

f = open('course_selections.pkl','rb')
selections = pickle.load(f)

f = open('student_acad_programs.pkl','rb')
students = pickle.load(f)

f = open('module_attrs.pkl', 'rb')
modules = pickle.load(f)

In [ ]:
# Construct mappings and relations. Split train, test sets. Feed those to hyperbolic embedding models. 

In [ ]:
selections['student <5240>']

In [ ]:
# Group graph data points (student-course/attributes, course-course/attributes) by year
from collections import defaultdict

# print(selections['student <12644>'])
years = defaultdict(list)

# years: 1000, 1100, 1200, ...
for (name, choices) in selections.items():
    for (year, course) in choices:
        years[year].append((name, 'enrolledin', course))


In [ ]:
# Append student attributes to the 'years' collection as well.
for (name, info) in students.items():
    for (year, major) in info:
        years[year].append((name, 'majoredin', major))

In [ ]:
years.keys()

In [ ]:
modules

In [ ]:
# Process modules information and append to 'years' collection.
for (mod, history) in modules.items():
    for (acadyear, info) in history.items():
        if 'SemesterName' in info and info['SemesterName']:
            year = acadyear[2:4] + info['SemesterName'][-1] + '0'
            if year.startswith('0'):
                year = year[1:]
            print(year)
            if year in years:
                for (attr, value) in info.items():
                    if attr not in ['ModuleCode', 'AcadYear', 'SemesterName', 'ModuleTitle', 'ModuleDescription'] and value not in ['Nil', "NIL", '--', '']:
                        print((mod, attr, value)) 
                        years[year].append((mod, attr, value))            
    #         break
    # break

In [7]:
# Given values for MRR, H@1, H@3, H@10 for GraIL-TransE and GraIL-AttH
grail_transE_values = [0.256, 0.185, 0.277, 0.397]  # GraIL-TransE
grail_attH_values = [0.271, 0.197, 0.293, 0.413]    # GraIL-AttH

# Calculate the percentage increase from GraIL-TransE to GraIL-AttH for each metric
percentage_increases = [
    ((attH - transE) / transE) * 100 for attH, transE in zip(grail_attH_values, grail_transE_values)
]

# Calculate the average percentage increase
average_percentage_increase = sum(percentage_increases) / len(percentage_increases)
average_percentage_increase

5.538065367984229

In [ ]:
# import pickle
# f = open('graph_unencoded_by_years.pkl', 'wb')
# pickle.dump(years, f)

In [ ]:
modules['AY1111']

In [ ]:
students

In [ ]:
years

In [ ]:
# Create mappings from students, courses, attributes to integers.
# Make distinct set of students and courses as well as their attributes.

# Handle pre/co-requisites
listofmods = sorted(list(modules.keys()))
mod2ind = dict(zip(listofmods, list(range(len(listofmods)))))
ind2mod = dict(zip(list(range(len(listofmods))), listofmods))

# Encode set of students
listofstdts = sorted(set(students.keys()))
stu2ind = dict(zip(listofstdts, list(range(len(listofmods), len(listofmods)+len(listofstdts)))))
ind2stu = dict(zip(list(range(len(listofmods), len(listofmods)+len(listofstdts))), listofstdts))

accu = len(listofmods) + len(listofstdts)

# Encode student/course attributes
majors = sorted(set([tpl[1] for enrols in students.values() for tpl in enrols]))
maj2ind = dict(zip(majors, list(range(accu, accu+len(majors)))))
ind2maj = dict(zip(list(range(accu, accu+len(majors))), majors))

accu += len(majors)
attrs = set()
for year in years.values():
    for tpl in year:
        if tpl[1] not in ['enrolledin', 'majoredin', 'Prerequisite','CrossModule','Preclusion','Corequisite']:
            attrs.add(tpl[2])

attrs = sorted(attrs)
atr2ind = dict(zip(attrs, list(range(accu, accu+len(attrs)))))
ind2atr = dict(zip(list(range(accu, accu+len(attrs))), attrs))

# General dictionary
graph2ind = {**stu2ind, **mod2ind, **maj2ind, **atr2ind}
ind2graph = {**ind2stu, **ind2mod, **ind2maj, **ind2atr}

import pickle
f1 = open('graph2ind.pkl', 'wb')
pickle.dump(graph2ind, f1)
f2 = open('ind2graph.pkl', 'wb')
pickle.dump(ind2graph, f2)

# Map relations to indices
rels = ['enrolledin', 'majoredin', 'Prerequisite',
        'CrossModule','Preclusion','Corequisite', 'Faculty',
        'Department', 'ModuleCredit', 'Workload']
rel2ind = dict(zip(rels, list(range(len(rels)))))
ind2rel = dict(zip(list(range(len(rels))), rels))


code_ptn = r"[a-zA-Z]{1,4}[0-9]{1,4}"
import re

def map_mod_idx_from_text(text):
    matches = re.findall(code_ptn, text)
    indices = []
    for match in matches:
        if match in mod2ind:
            indices.append(mod2ind[match])
    return indices
    
encoded_years = defaultdict(list)
unf_pkls = []

for (year, tpls) in years.items():
    n_tpls = []
    for tpl in tpls:
        tpl = list(tpl)
        tpl[0] = graph2ind[tpl[0]]
        tpl[1] = rel2ind[tpl[1]]
        if tpl[1] in [2, 3, 4, 5]:
            ids = map_mod_idx_from_text(tpl[-1])
            for id_ in ids:
                n_tpls.append((tpl[0], tpl[1], id_))
        elif tpl[2] in graph2ind:
            tpl[2] = graph2ind[tpl[2]]
            n_tpls.append(tuple(tpl))
        else:
            unf_pkls.append(tpl)
            # print(tpl)
            continue
    encoded_years[year].extend(n_tpls)

In [ ]:
unf_pkls[:100]

In [ ]:
rel2ind

In [4]:
## Save pickle
# import pickle
# f = open("../pickles/encoded_triplets.pkl", 'wb')
# pickle.dump(encoded_years, f)

## Load pickle
import pickle
f = open("../pickles/encoded_triplets.pkl", 'rb')
encoded_years = pickle.load(f)

graph = encoded_years['1710']

n1 = set([tpl[0] for tpl in graph])
n2 = set([tpl[2] for tpl in graph])
r = set([tpl[1] for tpl in graph])
n1.union(n2)
nnodes = len(n1)
print(nnodes)
print(r)
print(len(graph))


# For each student, we treat her next semester's selections as the correct answers.

29563
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
156355


In [6]:
# Define the values for GraIL-AttH and GraIL
grail_attH_values = [0.271, 0.197, 0.293, 0.413]
grail_values = [0.261, 0.186, 0.288, 0.399]

# Calculate the percentage increase from GraIL to GraIL-AttH for each metric
percentage_increases = [
    ((attH - grail) / grail) * 100 for attH, grail in zip(grail_attH_values, grail_values)
]
percentage_increases

[3.831417624521076, 5.913978494623661, 1.736111111111113, 3.50877192982455]

In [2]:
encoded_years.keys()

dict_keys(['1100', '1200', '1300', '1000', '1400', '1310', '1020', '1120', '1110', '1220', '1210', '1010', '1320', '1410', '1420', '1610', '1620', '1720', '1710', '1140', '1030', '1700', '1130', '1240', '1810', '1820', '1040', '1510', '1520', '1340', '1910', '1920', '1230', '1640', '1630', '1330', '1430', '1530', '1440', '1500', '2020', '2010', '2030', '2040', '1940', '1930', '1730', '1740', '1830', '1840', '1600', '1540', '1023', '1800', '1900', '2000', '930', '2110', '910', '920', '940'])

In [ ]:
# Figure out the train/test splits based on timeline.
# There are many ways to split the timeline.
# For each student, only final semester courses are included in the test set.
# We need to alter the loss computation to ensure that courses selected in the same semester won't be distinguished in their order.


In [ ]:
train = []
test = []
val = []
import random

for edge in graph:
    rel = edge[1]
    # edge = (edge[0], edge[2])
    if rel == 0:
        choice = random.randint(0,10)
        if choice == 0:
            test.append(edge)
        elif choice == 1:
            val.append(edge)
        else:
            train.append(edge)
    else:
        train.append(edge)

ftr = open('1710_train.txt', 'w')
fte = open('1710_test.txt', 'w')
fva = open('1710_val.txt', 'w')

ftr.writelines([' '.join(map(str, line)) + '\n' for line in train])
fte.writelines([' '.join(map(str, line)) + '\n' for line in test])
fva.writelines([' '.join(map(str, line)) + '\n' for line in val])

ftr.close()
fte.close()
fva.close()

In [1]:
# Train-test splits. Take an example of year 1710.
# Knowledge graph completion: complete what? edges.
# Only split based on student selections.
# Basic splits: 80% train, 10%-10% for valid/test
